## SDAE - Stacked Denoising Auto-Encoder

Generally, weights in a neural network (for classification) are initialized randomly. This works fine when we have a lot of labelled data. But what if we have a lot of data but only a fraction of it is labelled? A good example of this case is google images. We have a ton of data there, but only a tiny fraction of it is actually labelled. Can we classify all the data by training a network only on that fraction of labelled data?

In this experiment we demonstrate that initiaizing the network with pretrained SDAE weights boosts classification performance. In a stacked auto-encoder, every layer is trained individually to predict it's input. 

In [1]:
# Import the necessary libraries/modules
import numpy as np # for array operations
from keras.models import Model, Sequential # for defining the architectures
from keras.layers import Dense, Dropout, Input # layers for building the network
from keras.utils import to_categorical # to_categorical does one-hot encoding

tmp = np.load('cifar_pca_train.npz')
data = tmp['data']
labels = tmp['labels']

train_data = data[:10000]      # We'll use only 10000 out of 50000 samples for classification
train_labels = labels[:10000]

tmp = np.load('cifar_pca_test.npz')
test_data = tmp['data']
test_labels = tmp['labels']

# Converting labels into one-hot vectors for training. one-hot encoding is nothing but dummyfing
train_labels = to_categorical(train_labels, 10) 
test_labels = to_categorical(test_labels, 10)

print(train_data.shape)
print(train_labels.shape)
print(test_data.shape)
print(test_labels.shape)

Using TensorFlow backend.


(10000, 781)
(10000, 10)
(10000, 781)
(10000, 10)


### Training an MLP for classification through randomly initialized weights
#### Note:  We'll use only 10000 out of 50000 samples for classification


In [2]:
# training a simple one hidden layer MLP for classification task

mlp = Sequential()
mlp.add(Dropout(0.2, input_shape=(781,)))
mlp.add(Dense(1000, activation='sigmoid'))
mlp.add(Dropout(0.5))
mlp.add(Dense(800, activation='sigmoid'))
mlp.add(Dropout(0.5))
mlp.add(Dense(10, activation='softmax'))

nb_epoch = 20
batch_size = 32

mlp.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


history = mlp.fit(train_data, train_labels,
                    batch_size=batch_size,
                    epochs=nb_epoch,
                    validation_data=(test_data, test_labels))

Train on 10000 samples, validate on 10000 samples
Epoch 1/20
10000/10000 [==============================] - 12s - loss: 2.1390 - acc: 0.2258 - val_loss: 1.9035 - val_acc: 0.3286
Epoch 2/20
10000/10000 [==============================] - 13s - loss: 1.9158 - acc: 0.3104 - val_loss: 1.7796 - val_acc: 0.3750
Epoch 3/20
10000/10000 [==============================] - 13s - loss: 1.8410 - acc: 0.3472 - val_loss: 1.7603 - val_acc: 0.3798
Epoch 4/20
10000/10000 [==============================] - 12s - loss: 1.7906 - acc: 0.3636 - val_loss: 1.7729 - val_acc: 0.3734
Epoch 5/20
10000/10000 [==============================] - 12s - loss: 1.7452 - acc: 0.3820 - val_loss: 1.7249 - val_acc: 0.3900
Epoch 6/20
10000/10000 [==============================] - 12s - loss: 1.7131 - acc: 0.3982 - val_loss: 1.7220 - val_acc: 0.3935
Epoch 7/20
10000/10000 [==============================] - 12s - loss: 1.6812 - acc: 0.4108 - val_loss: 1.7413 - val_acc: 0.3845
Epoch 8/20
10000/10000 [==============================

### Train a two layer stacked autoencoder
#### Note:  We'll use all 50000 samples for training. Since no labels are required for autoencoder.

The way we do this is: 
1. Train an AE (autoencoder) with 1000 nodes on the original 781 features. So this will give 1000 encoded features.
2. Train another AE (autoencoder1) with 800 nodes on the above 1000 features.

The weights of the encoding layers of the above two netwroks can be used to initialize the MLP for classification .

In [3]:
# Training an autoencoder model on cifar-10 PCA reduced data

input_img = Input(shape=(781,))
crrpt_img = Dropout(0.5)(input_img)
encoded = Dense(1000, activation='sigmoid')(crrpt_img)
decoded = Dense(781, activation='linear')(encoded)

autoencoder = Model(input_img,decoded)

nb_epoch = 20
batch_size = 32

autoencoder.compile(optimizer='adam',
                    loss='mean_squared_error')

history = autoencoder.fit(data, data,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=(test_data, test_data))


autoencoder.save('SDAE_l1_model.h5')

Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 52s - loss: 0.5822 - val_loss: 0.2333
Epoch 2/20
50000/50000 [==============================] - 52s - loss: 0.5361 - val_loss: 0.2147
Epoch 3/20
50000/50000 [==============================] - 52s - loss: 0.5312 - val_loss: 0.2068
Epoch 4/20
50000/50000 [==============================] - 52s - loss: 0.5286 - val_loss: 0.2022
Epoch 5/20
50000/50000 [==============================] - 52s - loss: 0.5273 - val_loss: 0.2010
Epoch 6/20
50000/50000 [==============================] - 53s - loss: 0.5246 - val_loss: 0.2013
Epoch 7/20
50000/50000 [==============================] - 53s - loss: 0.5226 - val_loss: 0.1996
Epoch 8/20
50000/50000 [==============================] - 53s - loss: 0.5213 - val_loss: 0.1963
Epoch 9/20
50000/50000 [==============================] - 53s - loss: 0.5196 - val_loss: 0.1994
Epoch 10/20
50000/50000 [==============================] - 53s - loss: 0.5169 - val_lo

In [4]:
encoder = Model(input_img,encoded)
htrain_data = encoder.predict(data)

In [5]:
input_img1 = Input(shape=(1000,))
crrpt_img1 = Dropout(0.5)(input_img1)
encoded1 = Dense(800, activation='sigmoid')(crrpt_img1)
decoded1 = Dense(1000, activation='sigmoid')(encoded1)

autoencoder1 = Model(input_img1,decoded1)

autoencoder1.compile(optimizer='adam',
                    loss='binary_crossentropy')

history = autoencoder1.fit(htrain_data, htrain_data,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    shuffle=True)

autoencoder1.save('SDAE_l2_model.h5')

Epoch 1/20
50000/50000 [==============================] - 54s - loss: 0.5029    
Epoch 2/20
50000/50000 [==============================] - 53s - loss: 0.4836    
Epoch 3/20
50000/50000 [==============================] - 53s - loss: 0.4812    
Epoch 4/20
50000/50000 [==============================] - 54s - loss: 0.4803    
Epoch 5/20
50000/50000 [==============================] - 54s - loss: 0.4799    
Epoch 6/20
50000/50000 [==============================] - 55s - loss: 0.4796    
Epoch 7/20
50000/50000 [==============================] - 58s - loss: 0.4794    
Epoch 8/20
50000/50000 [==============================] - 57s - loss: 0.4792    
Epoch 9/20
50000/50000 [==============================] - 56s - loss: 0.4790    
Epoch 10/20
50000/50000 [==============================] - 55s - loss: 0.4788    
Epoch 11/20
50000/50000 [==============================] - 54s - loss: 0.4787    
Epoch 12/20
50000/50000 [==============================] - 55s - loss: 0.4786    
Epoch 13/20
50000/50000 [

In [7]:
#history.history.keys()
history.history['loss']

[0.50290534435272216,
 0.48362031801223754,
 0.4811713487434387,
 0.48032720893859865,
 0.47986385398864745,
 0.47960036675453188,
 0.47936453899383547,
 0.47915661866188047,
 0.47899607859611509,
 0.47882437093734742,
 0.47872783620834353,
 0.47856412561416628,
 0.47850920567512512,
 0.47841503306388855,
 0.4782769642353058,
 0.47818892166137694,
 0.4781111824798584,
 0.47803410091400145,
 0.4779452741241455,
 0.47785789219856262]

### Training an MLP for classification through autoencoder initialized weights

In [8]:
# Training with unsupervised initialization for layer-1 
# of an MLP using autoencoder weights

mlp = Sequential()
mlp.add(Dropout(0.2, input_shape=(781,)))
mlp.add(Dense(1000, activation='sigmoid'))
mlp.add(Dropout(0.5))
mlp.add(Dense(800, activation='sigmoid'))
mlp.add(Dropout(0.5))
mlp.add(Dense(10, activation='softmax'))

mlp.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])



In [9]:
mlp.layers

In [10]:
mlp.layers[1].set_weights(autoencoder.layers[2].get_weights())   # Import the encoder weights of autoencoder  
mlp.layers[3].set_weights(autoencoder1.layers[2].get_weights())  # Import the encoder weights of autoencoder1


history = mlp.fit(train_data, train_labels,
                    batch_size=batch_size,
                    epochs=nb_epoch,
                    validation_data=(test_data, test_labels))

Train on 10000 samples, validate on 10000 samples
Epoch 1/20
10000/10000 [==============================] - 13s - loss: 2.0550 - acc: 0.2583 - val_loss: 1.7527 - val_acc: 0.3948
Epoch 2/20
10000/10000 [==============================] - 12s - loss: 1.8333 - acc: 0.3506 - val_loss: 1.6883 - val_acc: 0.4084
Epoch 3/20
10000/10000 [==============================] - 12s - loss: 1.7409 - acc: 0.3734 - val_loss: 1.6507 - val_acc: 0.4184
Epoch 4/20
10000/10000 [==============================] - 13s - loss: 1.6593 - acc: 0.4159 - val_loss: 1.6161 - val_acc: 0.4284
Epoch 5/20
10000/10000 [==============================] - 13s - loss: 1.6043 - acc: 0.4305 - val_loss: 1.6052 - val_acc: 0.4281
Epoch 6/20
10000/10000 [==============================] - 13s - loss: 1.5490 - acc: 0.4594 - val_loss: 1.5852 - val_acc: 0.4397
Epoch 7/20
10000/10000 [==============================] - 12s - loss: 1.4992 - acc: 0.4717 - val_loss: 1.5853 - val_acc: 0.4378
Epoch 8/20
10000/10000 [==============================